In [1]:
import json
import os
import time
import pickle
import random

from instagramy import InstagramUser
from instagramy import InstagramHashTag
from instagramy import InstagramPost

from nedima.utils import adls_utils
from nedima.utils import groundswell as gs
from nedima.utils import env_setup


In [2]:
secrets = env_setup.load_secrets()
adls_client = adls_utils.initialize_storage_account_ad()
fs_client = adls_client.get_file_system_client(file_system=secrets['azure']['adls_fs_landing'])

In [3]:
#secrets

In [3]:
while(True):
    
    tag_dated, sleep_time = gs.start_inspection_iteration('surf')
    time.sleep(sleep_time)

    tag_latest = gs.inspect_posts('surf')

    dump_dict = gs.structure_inspection_json(tag_latest, tag_dated)
    adls_utils.dump_adls_json(dump_dict, tag_latest, fs_client)

    gs.dump_inspection_snapshot(tag_latest, tag_dated, 'surf')
    print("")

[SNAPSHOT] Next inspection scheduled for 2021-02-21 01:52:30. 0 seconds remaining.
[SESSION] Randomly selected the sessionid 45770346259%3AHjwhm9TUY86w2u%3A9 for the next inspection
[DIFF] 41 new posts in the last 474 seconds
[UPLOAD] posts_short was uploaded to the filesystem landing in the path tag/surf/posts_short/2021/02/21/015324_0041.json
[UPLOAD] posts_full was uploaded to the filesystem landing in the path tag/surf/posts_full/2021/02/21/015324_0041.json

[SNAPSHOT] Next inspection scheduled for 2021-02-21 02:02:38. 540 seconds remaining.
